In [13]:
from mistralai import Mistral
import yaml
import json
import requests 
from tqdm.auto import notebook_tqdm
from elasticsearch import Elasticsearch,  NotFoundError
import time
# from sentence_transformers import SentenceTransformer #embeddings
import pandas as pd

we are going to index the documents with the ids with elastic search. we are going to iterate over the question in our ground truth dataset (5 questions genrated for each question id)

for each q in question:
    execute q
    check if document is in the response

metrics: 
* Hit rate
* Mean reciprocal rank: the higher the position the relevant documents the better

In [2]:
with open('/workspaces/llm_search_engine/03_vector_search/docs_with_ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [3]:
es_client = Elasticsearch(
    hosts=[{'host': 'localhost', 'port': 9200, 'scheme': 'http'}],
    request_timeout=30,
    max_retries=10,
    retry_on_timeout=True
)

# Prueba la conexión
if es_client.ping():
    print("Conectado a Elasticsearch!")
else:
    print("No se pudo conectar a Elasticsearch.")

Conectado a Elasticsearch!


In [4]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
        }
    }
}

In [14]:
alias_name = "course-questions-metrics"

new_index = f"{alias_name}_{int(time.time()*1000)}"

In [16]:
try:
    es_client.indices.create(index=new_index, body=index_settings)
    print(f"Índice '{new_index}' creado exitosamente.")
except Exception as e:
    print(f"Error creando el índice '{new_index}': {e}")

actions = []
old_indices = {}
try:
    old_indices = es_client.indices.get_alias(name=alias_name)
    for idx in old_indices.keys():
        actions.append({"remove": {"index": idx, "alias": alias_name}})
except NotFoundError:
    print(f"No existía el alias '{alias_name}', se creará de nuevo.")

# Agregar la acción para añadir el alias al nuevo índice
actions.append({"add": {"index": new_index, "alias": alias_name}})

# Ejecutar la actualización de alias
try:
    es_client.indices.update_aliases(body={"actions": actions})
    print(f"Alias '{alias_name}' ahora apunta al índice '{new_index}'.")
except Exception as e:
    print(f"Error actualizando el alias: {e}")

Error creando el índice 'course-questions-metrics_1740737517601': BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions-metrics_1740737517601/P2vRCVe5Rsqu-YUNbxn58w] already exists')
Alias 'course-questions-metrics' ahora apunta al índice 'course-questions-metrics_1740737517601'.


In [8]:
index_name = "course-questions-metrics_index2"

if es_client.indices.exists(index=index_name):
    try:
        es_client.indices.delete(index=index_name, ignore_unavailable=True)
        print(f"Index {index_name} deleted successfully.")
    except Exception as e:
        print(f"Error deleting index {index_name}: {e}")

# Wait a short time before attempting to create the index again 
while es_client.indices.exists(index=index_name):
    print("Waiting for index to be fully deleted...")
    time.sleep(30)

try:
    es_client.indices.create(index=index_name, body=index_settings)
    print(f"Index {index_name} created successfully.")
except Exception as e:
    print(f"Error creating index {index_name}: {e}")

Index course-questions-metrics_index2 deleted successfully.
Error creating index course-questions-metrics_index2: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions-metrics_index2/OO-4TekTQ_qAsk9P2KNtwA] already exists')


In [19]:
cluster_health = es_client.cluster.health()
print(cluster_health)

{'cluster_name': 'docker-cluster', 'status': 'red', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 0, 'active_shards': 0, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 3, 'unassigned_primary_shards': 3, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 0.0}


In [20]:
print(es_client.cat.indices(format="json"))

[{'health': 'red', 'status': 'open', 'index': 'course-questions-metrics_1740737517601', 'uuid': 'P2vRCVe5Rsqu-YUNbxn58w', 'pri': '1', 'rep': '0', 'docs.count': None, 'docs.deleted': None, 'store.size': None, 'pri.store.size': None, 'dataset.size': None}, {'health': 'red', 'status': 'open', 'index': 'course-questions-metrics_index2', 'uuid': 'OO-4TekTQ_qAsk9P2KNtwA', 'pri': '1', 'rep': '0', 'docs.count': None, 'docs.deleted': None, 'store.size': None, 'pri.store.size': None, 'dataset.size': None}, {'health': 'red', 'status': 'open', 'index': 'course-questions-metrics_1740737242', 'uuid': '85fLtpfHQ_mR9PyDcBCVvg', 'pri': '1', 'rep': '0', 'docs.count': None, 'docs.deleted': None, 'store.size': None, 'pri.store.size': None, 'dataset.size': None}]


In [17]:
for doc in documents:
    es_client.index(index=new_index, document=doc)
    break

ConnectionTimeout: Connection timeout caused by: ConnectionTimeout(Connection timeout caused by: ReadTimeoutError(HTTPConnectionPool(host='localhost', port=9200): Read timed out. (read timeout=30)))

elastic search is giving many issues while configuring and setting indexes and indexing documents. In the begginng it worked fine, but not always. There are probably several reasons to this. One is that i use it in a virtual machine with docker and it doesnt have enough resources. The allocation of resources within the cluster seems a constant in all the log messages. The the definition and caching of indexes seem a big issue, not even deleting everything and setting it up again seem to help. Probaly due to cluster mis coordination and to problems with the docker configuration. Mybe my git hub space doesn't have enough memory either and that's one of the reason why this is not properly working. Since i don't want to waste more time on this im going to continue as it is, maybe trying for an alternative, or switching directly to a loca lconfiguration without docker.